<a href="https://colab.research.google.com/github/GruffGemini/ComputationalLinguistics/blob/main/%D0%94%D0%97_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1


In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 28.5 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 880 kB 57.5 MB/s 
     |████████████████████████████████| 6.6 MB 49.3 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=657025c993b0be3d8f8ddcdc7f3f7de3fbd756b046592bb87af58d2c18afe702
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

DEVICE = torch.device("cuda:0")

In [3]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

В качестве обучающей выборки возьмем небольшую коллекцию тостов (около 200 текстов по 5-10 предложений в каждом).

In [273]:
texts_path = 'drive/MyDrive/CompLing10/hw10.txt'
with open(texts_path) as f:
  texts = [i.strip() for i in f.readlines()]

In [274]:
texts

['Богатый бизнесмен решил жениться и обратился в брачную контору, чтобы ему подыскали даму из высшего общества. – Красивую, естественно? – поинтересовался брачный агент. – Конечно. Такую, чтобы о ней можно было сказать, что она как будто сошла с картины, – уточнил свое желание бизнесмен. После того, как бюро организовало ему встречу, он ворвался в брачную контору, брызгая слюной от возмущения: – Я же вас просил: красивую, как на картине. А кого вы мне подсовываете: уши, как лопухи, нос лезет на губу, глаза косые. – Сэр, не надо так шуметь, – обратился к нему один из агентов. – Это ведь дело вкуса. Кому нравится Рафаэль, а кто и без ума от Пикассо. Так выпьем же за разнообразие вкусов, о которых не спорят.',
 'Плывет по реке черепаха, на спине у нее сидит змея. Змея думает: – Укушу – сбросит! Черепаха думает: – Сброшу – укусит! Так выпьем же за верную женскую дружбу, которая помогает преодолевать все преграды!',
 'Сельский двор. В углу петух активно ухаживает за курочкой. А в это время 

In [5]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=texts_path, block_size=64, overwrite_cache=True) 
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [28]:
training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=200, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
trainer.train()

***** Running training *****
  Num examples = 345
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 200


Step,Training Loss


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=1.0742225646972656, metrics={'train_runtime': 1108.9433, 'train_samples_per_second': 62.221, 'train_steps_per_second': 0.18, 'total_flos': 2253643776000000.0, 'train_loss': 1.0742225646972656, 'epoch': 200.0})

In [ ]:
model.eval()

In [31]:
text = "Однажды"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=0.8, max_length=100)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Однажды он женился, и у него родился сын. Так выпьем же за то, чтобы женщины никогда не забывали о своем любимом деле!
Одна молодая грузинка приходит домой после праздника и видит перед собой молодую и красивую женщину. Она спрашивает: – Как тебе понравилась моя работа? – Это моя любовь, – отвечает молодая грузинка. – Почему ты так понравилась мне? – Это моя жена, которая всегда со мной. Так выпьем же за женскую любовь!
Молодая грузинка приходит домой


In [48]:
text = "Однажды в студеную"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=0.8, max_length=150)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Однажды в студеную заснеженную ночь к нему в гости пришел друг и предложил напиться. Студент согласился, но только одну бутылку. Расставаясь с друзьями, друг сказал: – Если бы мы жили сейчас в такой дали, я бы дал тебе напиться. Студент подумал и ответил: – Если бы я жил сейчас в такой дали, я бы подарил тебе бутылку коньяка. – А если бы я хотела вернуться домой только завтра, – сказала жена, – ты бы меня отпустил? – Не отпустил бы, – ответил друг, – но зато сейчас ты можешь меня отвезти в далекие края на своей машине. Так выпьем же за то, чтобы ни одна поездка в далекие края не была для нас стóлькими перелетами!
Встре


In [214]:
text = "В Москве"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=0.5, max_length=200)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



В Москве проходит учредительная конференция Ассоциации. В ней принимают участие более 100 известных московских компаний. В этом году на торжественной церемонии открытия конференции генеральный директор «АВТОВАЗа» подвел итоги работы своей компании за прошедший год. – За последние три года мы сумели увеличить свой штат, – говорит он. – И мы гордимся тем, что являемся одной из крупнейших в стране компаний-лидеров в производстве грузовых автомобилей. Так давайте выпьем за успех нашей совместной работы!
В одном из деловых разговоров один из собеседников заметил: – Я бы хотел, чтобы наша встреча стала настоящим праздником!
Однажды в поезде Вано Мато посетил ресторан. Заведение было расположено в старинном здании, окна которого были завешаны картинами. Заведение было переполнено, и в одном углу зала Вано Мато увидел объявление: «Шаббат». – Что это значит? – спросил Вано. – Это значит, что ты можешь исполнить любое желание хозяина, – ответил администратор. – А что это


In [272]:
text = "Недвижимость снова"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=1, top_k=30, max_length=200)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Недвижимость снова подорожала – теперь ее стоимость может возрасти до нескольких тысяч рублей! И это вполне естественно – рынок загородной недвижимости активно развивается, а значит, и цены на недвижимость тоже должны быть не ниже, чем у конкурентов! Предлагаю поднять наш бокал за то, чтобы загородная недвижимость всегда была доступна каждому!
Жилища вилл и домов на берегу моря всегда вызывали у гостей особенную историю. Если вы любите спокойный и размеренный отдых, то вам точно понравятся виллы на Средиземном море, в которых вы сможете насладиться прекрасным видом на море. Конечно, это не Египет, но все же. Вилла на Средиземном море, где вы в спокойной обстановке можете отдохнуть от городской суеты и забыть обо всех проблемах, является отличным решением для тех, кто живет в городе. Вилла на Средиземном море – это, пожалуй, самое большое и самое красивое на земле чудо природы! И если ваш первый отдых на море был просто сказкой, то сейчас вы можете окунуться в царящую в


In [100]:
text = "Вчера внезапно"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=0.6, top_k=20, max_length=150)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вчера внезапно узнала, что мой муж изменяет моей жене. И что я должна его пристыдить. Позвала я тогда свою жену и сказала: – Ну ладно, я его понимаю, он не ангел. Но скажи мне, жена, сколько ты должна заплатить за его измену? – Ну, это уже другой вопрос. Если бы ты знала, как я его люблю! – Ну что же, хорошо, я его понимаю. Но скажи мне, жена, сколько ты должна заплатить за его измену? – Ну, это уже другой вопрос. Если бы я знала, сколько ты должна заплатить за его измену, то сказала бы тебе сразу. Так выпьем же за женскую логику!
Встретились два кавказца. Один другому говорит: – Я понимаю


In [188]:
text = "На следующий день"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = model.generate(input_ids, do_sample=True, temperature=0.5, top_k=15, max_length=200)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



На следующий день после свадьбы муж приходит домой и говорит своей жене: – Я развожусь со своей женой. – Почему ты так говоришь? – Да так, просто жена у меня умница. А ты знаешь, что я умный? Так выпьем же за то, чтобы женщины не спорили из-за каких-нибудь пустяков!
На Кавказе есть обычай: если жена изменяет мужу, она должна выйти замуж за своего соседа. Так вот, когда я женился, у меня не было ни единой ссоры, и у меня не было повода для ревности. Так выпьем же за то, чтобы женщины никогда не спорили из-за пустяков!
Однажды в горных ущельях Кавказа шли двое: молодой и старый. Молодой шел впереди, а девушка шла позади. Когда они оказались в одной горной деревушке, старик сказал юноше: – Пойдем со мной, я покажу тебе твои ущелья. – Но у меня нет ни деревца, ни дороги, – испугался юноша. – Я стар, мне не по


# Задание 2

1) GPT-1 была описана в работе "Improving Language Understanding
by Generative Pre-Training" (Radford, et al., 2018)

GPT-2 -  в работе "Language Models are Unsupervised Multitask Learners" (Radford, et al., 2019)

GPT-3 - в статье "Language Models are Few-Shot Learners" (Brown, et al., 2020)

2) Обучающий корпус для GPT-3 состоял из 5 частей: Common Crawl, WebText2, 2 книжных корпуса и Википедия. Особенностью обучения являлось то, что более качественные части корпусов имели больший вес при обучении, и отрывки из них использовались чаще. Так, например, корпус Common Crawl имел вес 60%, WebText2 - 22%, все остальные - менее 10%. Таким образом создатели пытались улучшить общее качество модели.